In [57]:
import numpy as np
from sklearn.svm import SVC
import pandas as pd
import requests
import os
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [46]:
url_data = 'https://raw.githubusercontent.com/christian-igel/ML/main/data/NFI_filtered_cleaned.csv'
fn = 'ML_filtered_cleaned.csv'

if not os.access(fn, os.R_OK):
    print('downloading', url_data)
    try:
        r = requests.get(url_data)
        r.raise_for_status()
        open(fn, 'wb').write(r.content)
    except Exception as e:
        print('download failed:', e)

df = pd.read_csv(fn)

In [47]:
regression_target = "BMag_ha"
classification_target = "C_frac"
features = [
    'h_mean_1_', 'h_mean_2_', 'h_std_1_', 'h_std_2_', 'h_coov_1_', 'h_coov_2_', 'h_skew_1_', 'h_skew_2_',
    'IR_', 'h_q5_1_', 'h_q10_1_', 'h_q25_1_', 'h_q50_1_', 'h_q75_1_', 'h_q90_1_', 'h_q95_1_', 'h_q99_1_',
    'h_q5_2_', 'h_q10_2_', 'h_q25_2_', 'h_q50_2_', 'h_q75_2_', 'h_q90_2_', 'h_q95_2_', 'h_q99_2_', 
    'red_q75', 'red_q50', 'red_q25', 'blue_q75', 'blue_q50', 'blue_q25', 'green_q75', 'green_q50', 'green_q25'
]


In [48]:
df_trainval = df.query("split in ['train', 'val']")
df_test = df.query("split == 'test'")

X_trainval = df_trainval[features]
y_trainval = df_trainval[[regression_target]]
y_trainval = y_trainval.values.ravel() # i use ravel() to flatten the 2D array into 1D array in order to  remove the warnings i get in the 2. task below

X_test = df_test[features]
y_test = df_test[[regression_target]]
y_test = y_test.values.ravel() 

In [49]:
def compute_upper_bound_on_generalization_error_rf(model, X, y):

    # Compute the number of possible predictions that the rf can make
    num_predictions = 2 ** len(X_trainval.columns)

    # Compute the generalization error bound
    generalization_error_bound = np.sqrt((np.log(num_predictions) + np.log(2.0)) / (2 * len(X)))

    return generalization_error_bound

In [50]:
#This is only if you want to use rademacher instead. 
def compute_rademacher_complexity(model, X, y):

    # Generate random Rademacher variables for each data point in the training set.
    rademacher_variables = np.random.rand(len(X)) * 2 - 1

    # Train the SVM on the training data, with the labels multiplied by the Rademacher variables.
    model.fit(X, y * rademacher_variables)

    # Compute the error of the SVM on the training data.
    error = model.score(X, y)

    return error

def compute_upper_bound_on_generalization_error(model, X, y):

    # Compute the Rademacher complexity of the SVM.
    rademacher_complexity = compute_rademacher_complexity(model, X, y)

    # Compute the generalization error bound.
    generalization_error_bound = rademacher_complexity * np.sqrt(np.log(2) / (2 * len(X)))

    return generalization_error_bound

In [51]:
# Define the range of hyperparameters to try
n_estimators = [100, 1000]
max_depth = np.linspace(1, 10, 10)

# Cast the `max_depth` and 'n_estimators' parameter to an integer
max_depth = max_depth.astype(int)
#n_estimators = n_estimators.astype(int)

# Train the random forest classifier with different hyperparameters
random_forests = []
for n_estimator in n_estimators:
    print(f"Next n_estimator: {n_estimator}")
    for depth in max_depth:
        print(f"Next depth: {depth}")
        rf = RandomForestRegressor(n_estimators=n_estimator, max_depth=depth)
        rf.fit(X_trainval, y_trainval)
        random_forests.append(rf)
print(f"Done {random_forests}")

# Compute the upper bound on the generalization error for each hyperparameter setting
upper_bounds = []
for rf in random_forests:
    print(f"Next rf: {rf}")
    upper_bound = compute_upper_bound_on_generalization_error_rf(rf, X_trainval, y_trainval)
    print(f"Done {upper_bound}")
    upper_bounds.append(upper_bound)
print(f"Done {upper_bounds}")

# Select the hyperparameter setting that minimizes the upper bound on the generalization error
n_estimator_opt = n_estimators[np.argmin(upper_bounds)]
max_depth_opt = max_depth[np.argmin(upper_bounds)]

# Train the random forest classifier with the optimal hyperparameters
rf_opt = RandomForestRegressor(n_estimators=n_estimator_opt, max_depth=max_depth_opt)
rf_opt.fit(X_trainval, y_trainval)

Next n_estimator: 100
Next depth: 1
Next depth: 2
Next depth: 3
Next depth: 4
Next depth: 5
Next depth: 6
Next depth: 7
Next depth: 8
Next depth: 9
Next depth: 10
Next n_estimator: 1000
Next depth: 1
Next depth: 2
Next depth: 3
Next depth: 4
Next depth: 5
Next depth: 6
Next depth: 7
Next depth: 8
Next depth: 9
Next depth: 10
Done [RandomForestRegressor(max_depth=1), RandomForestRegressor(max_depth=2), RandomForestRegressor(max_depth=3), RandomForestRegressor(max_depth=4), RandomForestRegressor(max_depth=5), RandomForestRegressor(max_depth=6), RandomForestRegressor(max_depth=7), RandomForestRegressor(max_depth=8), RandomForestRegressor(max_depth=9), RandomForestRegressor(max_depth=10), RandomForestRegressor(max_depth=1, n_estimators=1000), RandomForestRegressor(max_depth=2, n_estimators=1000), RandomForestRegressor(max_depth=3, n_estimators=1000), RandomForestRegressor(max_depth=4, n_estimators=1000), RandomForestRegressor(max_depth=5, n_estimators=1000), RandomForestRegressor(max_depth

RandomForestRegressor(max_depth=1)

In [54]:
y_pred_randomF = rf_opt.predict(X_test)
r2_randomF = r2_score(y_test, y_pred_randomF)
print(f"Best Random Forest R^2 Score: {r2_randomF}")

Best Random Forest R^2 Score: 0.5634193875213307


In [59]:
# Define a grid of hyperparameters to try
param_grid = {
    'n_estimators': [100, 1000],
    'max_depth': [5, 10, 15]
}

# Create a grid search object
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, verbose=2)

# Fit the grid search object to the training data
grid_search.fit(X_trainval, y_trainval)

# Get the best random forest
best_rf = grid_search.best_estimator_

# Compute the training error of the random forest
training_error = best_rf.score(X_trainval, y_trainval)

# Compute the generalization error bound
generalization_error_bound = np.sqrt(np.log(2) / (2 * len(X_trainval.columns))) + training_error

# Print the generalization error bound
print("Generalization error bound:", generalization_error_bound)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ......................max_depth=5, n_estimators=100; total time=   1.8s
[CV] END ......................max_depth=5, n_estimators=100; total time=   1.7s
[CV] END ......................max_depth=5, n_estimators=100; total time=   1.8s
[CV] END ......................max_depth=5, n_estimators=100; total time=   1.8s
[CV] END ......................max_depth=5, n_estimators=100; total time=   1.7s
[CV] END .....................max_depth=5, n_estimators=1000; total time=  17.5s
[CV] END .....................max_depth=5, n_estimators=1000; total time=  17.4s
[CV] END .....................max_depth=5, n_estimators=1000; total time=  17.5s
[CV] END .....................max_depth=5, n_estimators=1000; total time=  17.5s
[CV] END .....................max_depth=5, n_estimators=1000; total time=  17.4s
[CV] END .....................max_depth=10, n_estimators=100; total time=   3.3s
[CV] END .....................max_depth=10, n_est

In [60]:
y_pred_randomF = best_rf.predict(X_test)
r2_randomF = r2_score(y_test, y_pred_randomF)
print(f"Best Random Forest R^2 Score: {r2_randomF}")

Best Random Forest R^2 Score: 0.7127832927368524
